#model

In [41]:
import pandas as pd

In [42]:
chattogram_df = pd.read_csv("https://raw.githubusercontent.com/DryBoss/diarrhea-prediction-model/main/data/Chattogram.csv")
dhaka_df = pd.read_csv("https://raw.githubusercontent.com/DryBoss/diarrhea-prediction-model/main/data/Dhaka.csv")
khulna_df = pd.read_csv("https://raw.githubusercontent.com/DryBoss/diarrhea-prediction-model/main/data/Khulna.csv")
rajshahi_df = pd.read_csv("https://raw.githubusercontent.com/DryBoss/diarrhea-prediction-model/main/data/Rajshahi.csv")

In [43]:
# List of dataframes to process
dataframes = [chattogram_df, dhaka_df, khulna_df, rajshahi_df]

# Process each dataframe
for df in dataframes:
    # Ensure 'Date' is in datetime format
    df['Date'] = pd.to_datetime(df['Date'])

    # Smooth each column (excluding 'Date') using a 30-day moving average
    for column in df.columns:
        if column != 'Date':  # Skip the 'Date' column
            smoothed_column = f'{column}_smooth'
            df[smoothed_column] = df[column].rolling(window=30).mean()
